In [3]:
import pandas as pd
import glob

# Läs in alla Excel-filer
files = glob.glob('../data/original_excels/*.xlsx')

dfs = []
for file in files:
    df = pd.read_excel(file)
    df['Källa_fil'] = file  # valfritt: spåra vilken fil den kom från
    dfs.append(df)

# Slå ihop allt till en DataFrame
all_data = pd.concat(dfs, ignore_index=True)

In [ ]:
all_data.info()
all_data['Djurslag'].value_counts()
all_data['Datum'] = pd.to_datetime(all_data['Datum'], errors='coerce')
all_data['Månad'] = all_data['Datum'].dt.month
all_data['Veckodag'] = all_data['Datum'].dt.day_name()
